In [1]:
import autograd.numpy as np
import autograd.scipy as scipy
# import numpy as np
# import scipy
import scipy.linalg
import scipy.cluster
import scipy.stats
from autograd import grad, elementwise_grad, jacobian, hessian, value_and_grad
from collections import OrderedDict
import random

import plotly
from plotly.offline import iplot as plt
from plotly import graph_objs as plt_type
plotly.offline.init_notebook_mode(connected=True)

from IPython.core.debugger import set_trace



/nfs/nhome/live/gbohner/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



# Kernel function + derivatives

In [2]:
def square_dist(x, x2=None, lengthscales=None):
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    
    x = x / lengthscales
    xs = np.sum(np.square(x), axis=0)
    if x2 is None:
        return -2 * np.dot(x.T, x) + \
               np.reshape(xs, (-1, 1)) + np.reshape(xs, (1, -1))
    else:
        x2 = x2 / lengthscales
        x2s = np.sum(np.square(x2), axis=0)
        return -2 * np.dot(x.T, x2) + \
               np.reshape(xs, (-1, 1)) + np.reshape(x2s, (1, -1))

def euclid_dist(x, x2, lengthscales=None):
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    r2 = square_dist(x, x2, lengthscales)
    return np.sqrt(r2 + 1e-12)

def RBF(x, x2=None, lengthscales=None, kernel_variance=1.):
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((x.shape[1], 0))
    
    if lengthscales is None:
        lengthscales=np.ones((x.shape[0], 1))
    
    return kernel_variance*np.exp(-square_dist(x, x2, lengthscales=lengthscales)/2)

def dRBF(x, x2=None, *args,**kwargs):
    D = x.shape[0]
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((D*x.shape[1], 0))
        
    if x2 is None:
        x2 = x
        
    N_x1 = x.shape[1]
    N_x2 = x2.shape[1]
    
#     # Get kernel matrix (N_x1 x N_x2)
#     if 'Kxx' in kwargs:
#         Kxx = kwargs['Kxx']
#     else:
#         Kxx = RBF(x, x2=x2, *args, **kwargs)
    
#     # Get pairwise distances per columns (D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs = x[:,:,None] - x2[:,:,None].swapaxes(1,2)
    
# #     # Elementwise version for testing
# #     out = np.zeros((D,N_x1,N_x2))
# #     for i in range(N_x1):
# #         for j in range(N_x2):
# #             out[:,i,j] = -(1./np.squeeze(lengthscales**2))*(x[:,i]-x2[:,j]) *Kxx[i,j]
    
#     out = - (1./(np.expand_dims(lengthscales**2,axis=2))) * ( XminusX2_pairwise_diffs * np.expand_dims(Kxx, axis=0) );
#     # We want to stack it to 2D to have shape (D*N_x1,  N_x2)
#     return np.reshape(out, (D*N_x1,N_x2), order='F')

    ### There is some discrepency here between the autograd dRBF and the above manual one, Test later?
    # The jacobian returns a shape
    # (N_x1, N_x2, D, 1)
    jRBF = jacobian(RBF)
    gradRBF = np.concatenate([jRBF(x[:,i:(i+1)], x2, *args, **kwargs) for i in range(N_x1)], axis=0)
    # For every x1 input point, compute a 1xN_x2xD jacobian, then stack them by point getting N_x1, N_x2, D, 1
    
    # We want to stack it to 2D to have shape 
    # (D*N_x1,  N_x2)
    
    # Here the derivative is with respect to the first argument, and it is ANTI-SYMMETRIC (Transpose -> minus sign)
    return np.reshape(gradRBF.swapaxes(1,2).swapaxes(0,1), (D*N_x1, -1), order='F')
    
    
def ddRBF(x, x2=None, *args, **kwargs):
    D = x.shape[0]
    if x.shape[1]==0:
        if (x2 is not None):
            return np.zeros((0, D*x2.shape[1]))
        else:
            return np.zeros((0,0))
    elif (x2 is not None):
        if x2.shape[1]==0:
            return np.zeros((D*x.shape[1], 0))
        
    if x2 is None:
        x2 = x
    
    N_x1 = x.shape[1]
    N_x2 = x2.shape[1]
    
#     # Get kernel matrix (N_x1 x N_x2)
#     if 'Kxx' in kwargs:
#         Kxx = kwargs['Kxx']
#     else:
#         Kxx = RBF(x, x2=x2, *args, **kwargs)
        
#     # Get pairwise distances per columns (D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs = x[:,:,None] - x2[:,:,None].swapaxes(1,2)
    
#     XminusX2_pairwise_diffs_rescaled = (1./(lengthscales**2)[:,:,None])*XminusX2_pairwise_diffs
    
#     # Get the outer product of pairwise distance per instance (D x D x N_x1 x N_x2)
#     XminusX2_pairwise_diffs_rescaled_outer = (np.expand_dims(XminusX2_pairwise_diffs_rescaled,axis=1)
#                                         * np.expand_dims(XminusX2_pairwise_diffs_rescaled,axis=0)
#                                      )
    
#     lengthscales_times_id = np.diag(np.squeeze(1./(lengthscales**2)))[:,:,None,None]
        
    
    
#     out = ((lengthscales_times_id - XminusX2_pairwise_diffs_rescaled_outer) 
#            * np.expand_dims(np.expand_dims(Kxx, axis=0),axis=0)
#            )
    
#     # Out has a shape of D x D x N_x1 x N_x2
#     # We want to stack it to 2D to have shape (D*N_x1, D*N_x2)
#     return np.reshape(out.swapaxes(1,2), (D*N_x1, D*N_x2), order='F')
    
    # The hessian defined here returns a shape
    # (D*N_x1, N_x2, D, 1)
    hRBF = jacobian(dRBF, argnum=1)
    
    hessRBF = np.concatenate([hRBF(x, x2[:,j:(j+1)], *args, **kwargs) for j in range(N_x2)], axis=1)
    
    # We want to stack it to 2D to have shape 
    # (D*N_x1, D*N_x2)
    
    return np.reshape(hessRBF.swapaxes(1,2), (D*N_x1, -1), order='F')


### Expected kernels for noisy input
Compute various kernel expectations of the RBF kernel written above, 
given the first kernel argument is a single Dx1 vector, given by mean $\mu$ (Dx1) and diagonal variance $\sigma$ (Dx1)

Note: Lengthscales are sqrt(gaussian_covariance)

In [14]:
dRBF(x2,x1).shape

(30, 3)

In [15]:
dRBF(x1,x2).shape

(15, 6)

In [18]:
x1 = np.random.randn(5,1)
x2 = np.random.randn(5,6)
dRBF(x1,x2).shape

(5, 6)

In [31]:
np.allclose(dRBF(x2,x1).reshape(6,5).transpose() , (-dRBF(x1,x2)))

True

In [3]:
# # Testing noisy (with little noise) vs non-noisy versions
mu = np.array([[0], [[0]]])
sigma = np.array([[1e-1], [1e-1]])
lengthscales = np.array([[1.0], [1.0]])
kernel_variance = 1.0
X = np.concatenate([np.arange(-5,5,0.5)[:,None].T, np.arange(-5,5,0.5)[:,None].T])


In [5]:

def RBF_eK(mu, sigma, X, lengthscales=None, kernel_variance=1):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ k(x, X)], a 1 x M array
    """
    if X.shape[1]==0:
        return np.zeros((1, 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
    return RBF(x=mu, 
               x2=X, 
               lengthscales=np.sqrt(lengthscales**2 + sigma), 
               kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod(lengthscales**2 + sigma))
              )


In [9]:
def RBF_exK(mu, sigma, X, lengthscales=None, kernel_variance=1, eK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ x * k(x, X)], a D x M array
    """
    
    if X.shape[1]==0:
        return np.zeros((x.shape[0], 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    if eK is None:
        eK = RBF_eK(mu, sigma, X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    mean_gauss = (X/(lengthscales**2) + mu/sigma)/(1/(lengthscales**2)+(1/sigma))
    
    return eK*mean_gauss

print(RBF_exK(np.array([[1.5], [[1]]]), np.array([[1e-20], [1e-2]]), X[:,11:14]))

print(np.array([[1.5], [[1]]]) * RBF(np.array([[1.5], [[1]]]), X[:,11:14]))

[[0.7998969  1.31717586 1.31880703]
 [0.53062468 0.87811724 0.88355719]]
[[0.80289214 1.32374535 1.32374535]
 [0.53526143 0.8824969  0.8824969 ]]


In [ ]:
def RBF_edK(mu, sigma, X, lengthscales=None, kernel_variance=1, eK=None, exK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ dk(x, X) ], an 1 x (D x M) array
    We want it differentiated with respect to the second argument X -> No minus sign (minus signs cancel)
    """
    if X.shape[1]==0:
        return np.zeros((1, 0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    if eK is None:
        eK = RBF_eK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    if exK is None:
        exK = RBF_exK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    
    return np.reshape((exK - X * eK)/(lengthscales**2),(1,-1), order='F')

In [11]:
def RBF_eKK(mu, sigma, X, lengthscales=None, kernel_variance=1):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [k(X, x) * k(x, X) ], an M x M array
    """
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
        
    kXX_scaled = RBF(
                       x=X, 
                       x2=X, 
                       lengthscales=np.sqrt(2*(lengthscales**2)), 
                       kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod(2*(lengthscales**2)))
                    )
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    kXpX_mu = RBF(
                       x=np.reshape(X_pairwise_sums/2,(mu.shape[0], -1), order='F'), 
                       x2=mu, 
                       lengthscales=np.sqrt((lengthscales**2)/2 + sigma), 
                       kernel_variance=kernel_variance*np.sqrt(np.prod(lengthscales**2)/np.prod((lengthscales**2)/2 + sigma))
                    )
    
    out = kXX_scaled * np.reshape(kXpX_mu, (X.shape[1], X.shape[1]), order='F')
    
    # Due to numerical instability, this is not always symmetric, fix:
    out = (out + out.T) / 2.
    
    return out

In [12]:
# # Testing noisy expected kernel outer product vs noiseless 
RBF_eKK(np.array([[0], [[0]]]), np.array([[1e-13], [1e-13]]), X[:,9:12]) - \
    RBF(np.array([[0], [[0]]]), X[:,9:12]) * RBF(np.array([[0], [[0]]]), X[:,9:12]).T

array([[-6.06181771e-14, -1.36335387e-13, -1.21236354e-13],
       [-1.36335387e-13, -2.00062189e-13, -1.36335387e-13],
       [-1.21236354e-13, -1.36335387e-13, -6.06181771e-14]])

In [13]:
def RBF_exKK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [x * k(X, x) * k(x, X) ], a D x M x M array
    """
    if X.shape[1]==0:
        return np.zeros((x.shape[0], 0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    
    # M x M array
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    # D x M x M array
    mean_gauss = ((X_pairwise_sums/2)/(((lengthscales**2)/2)[:,:,None]) + (mu/sigma)[:,:,None])/(
                                                (1/((lengthscales**2)/2)+(1/sigma))[:,:,None])
    
    
    return eKK[:,:,None].swapaxes(1,2).swapaxes(0,1) * mean_gauss

In [18]:
# Testing noisy expected kernel outer product

In [19]:
RBF_exKK(np.array([[1], [[1]]]), np.array([[1e-13], [1e-13]]), X[:,9:12])

array([[[0.011109  , 0.03877421, 0.082085  ],
        [0.03877421, 0.13533528, 0.2865048 ],
        [0.082085  , 0.2865048 , 0.60653066]],

       [[0.011109  , 0.03877421, 0.082085  ],
        [0.03877421, 0.13533528, 0.2865048 ],
        [0.082085  , 0.2865048 , 0.60653066]]])

In [20]:
RBF(np.array([[1], [[1]]]), X[:,9:12]) * RBF(np.array([[1], [[1]]]), X[:,9:12]).T

array([[0.011109  , 0.03877421, 0.082085  ],
       [0.03877421, 0.13533528, 0.2865048 ],
       [0.082085  , 0.2865048 , 0.60653066]])

In [21]:
def RBF_exxKK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [ (x*x.T) * k(X, x) * k(x, X) ], a D x D x M x M array
    """
    if X.shape[1]==0:
        return np.zeros((x.shape[0], x.shape[0], 0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
    
    # M x M array
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    # D x D array
    var_gauss = 1/(1/((lengthscales**2)/2)+(1/sigma))
    
    
    X_pairwise_sums = X[:,:,None] + X[:,:,None].swapaxes(1,2)

    # D x M x M array
    mean_gauss = ((X_pairwise_sums/2)/(((lengthscales**2)/2)[:,:,None]) + (mu/sigma)[:,:,None])*(var_gauss[:,:,None])
    
    # D x D x M x M array
    mean_outer = np.expand_dims(mean_gauss, axis=1) * np.expand_dims(mean_gauss, axis=0)
    
    return np.expand_dims(np.expand_dims(eKK, axis=0), axis=0) * (var_gauss[:,:,None,None] + mean_outer)



In [22]:
# # Testing noisy expected kernel outer product
RBF_exxKK(np.array([[1], [[1]]]), np.array([[1e-2], [1e-2]]), X[:,11:14])[0,1,:,:]

array([[0.59466685, 0.76542226, 0.60046769],
       [0.76542226, 0.99000384, 0.78043018],
       [0.60046769, 0.78043018, 0.61821573]])

In [23]:
RBF(np.array([[1], [[1]]]), X[:,11:14]) * RBF(np.array([[1], [[1]]]), X[:,11:14]).T

array([[0.60653066, 0.77880078, 0.60653066],
       [0.77880078, 1.        , 0.77880078],
       [0.60653066, 0.77880078, 0.60653066]])

In [26]:
def RBF_eKdK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None, exKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [  k(X, x) * dk(x, X)  ], an M x (D x M) array
    """
    
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
        
        
        
    # m1 x m2
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
        
    # d x m1 x m2
    if exKK is None:
        exKK = RBF_exKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)    
    
    
    # d x m1 x m2,
    # As exKK naturally uses the first argument and
    # X is the second argument in the derivative kernel, we should expand it, such that we iterate along m2 dimension    
    eKdK = (exKK - np.expand_dims(X, axis=1) * np.expand_dims(eKK, axis=0))/((lengthscales**2)[:,:,None])
       
    # We then finally modify the order of axis and the dimensionality to get 
    # the expected m1 - d - m2 order with M x (DM) shape
    
    return np.reshape(eKdK.swapaxes(0,1), (X.shape[1], -1), order='F')

In [27]:
# Testing
RBF_eKdK(np.array([[0], [[1]]]), np.array([[1e-20], [1e-20]]), X[:,13:17])[0:2,4:]



array([[-0.01021693, -0.00613016, -0.00129223, -0.00086149],
       [-0.0029272 , -0.00175632, -0.00037023, -0.00024682]])

In [28]:
RBF(np.array([[0], [[1]]]), X[:,13:15]).T * np.reshape(dRBF(X[:,15:17], np.array([[0], [[1]]])),(-1), order='F').T

array([[-0.01021693, -0.00613016, -0.00129223, -0.00086149],
       [-0.0029272 , -0.00175632, -0.00037023, -0.00024682]])

In [29]:
def RBF_edKdK(mu, sigma, X, lengthscales=None, kernel_variance=1, eKK=None, exKK=None, exxKK=None):
    """
    x ~ N(mu, sigma), Dx1
    X is DxM
    Return E_x [  dk(X, x) * dk(x, X)  ], a (D x M) x (D x M) array
    """
    if X.shape[1]==0:
        return np.zeros((0,0))
    
    if lengthscales is None:
        lengthscales=np.ones((mu.shape[0], 1))
            
    # m1 x m2
    if eKK is None:
        eKK = RBF_eKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance)
    
    # d1 x m1 x m2
    if exKK is None:
        exKK = RBF_exKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance,
                       eKK = eKK)
    
    # d1 x d2 x m1 x m2
    if exxKK is None:
        exxKK = RBF_exxKK(mu=mu, sigma=sigma, X=X, lengthscales=lengthscales, kernel_variance=kernel_variance,
                        eKK = eKK)
    
    
    
    
    edKdK = (exxKK # exKK
        -1.0 * np.expand_dims(np.expand_dims(X, axis=2), axis=1) * np.expand_dims(exKK, axis=0)  # X[:,None,:,None]
        -1.0 * np.expand_dims(np.expand_dims(X, axis=1), axis=0) * np.expand_dims(exKK, axis=1)  # X[None,:,None,:]
        + np.expand_dims(np.expand_dims(X, axis=2), axis=1) * np.expand_dims(np.expand_dims(X, axis=1), axis=0) 
           * np.expand_dims(np.expand_dims(eKK, axis=0), axis=0) # X[:,None,:,None] * X[None,:,None,:] * eKK[None,None,:,:]
    )
    
    # Divide with lengthscales appropriately
    edKdK = edKdK / ((lengthscales.T**2)[:,:,None,None])
    edKdK = edKdK / ((lengthscales**2)[:,:,None,None])
    
    # We then finally modify the order of axis and the dimensionality to get 
    # the expected m1 - d - m2 order with M x (DM) shape
    
    out = np.reshape(edKdK.swapaxes(1,2), (X.shape[0]*X.shape[1], X.shape[0]*X.shape[1]), order='F')
    
    # Due to numerical instability (TODO: Double check if really instab), this is not always symmetric, fix:
    out = (out + out.T) / 2.
    
    return out

In [30]:
# Testing
RBF_edKdK(np.array([[0.5], [[0]]]), np.array([[1e-20], [1e-20]]), X[:,11:14])

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.1947002 , 0.11809164, 0.23618328, 0.08688697,
        0.13033046],
       [0.        , 0.11809164, 0.0716262 , 0.1432524 , 0.05269961,
        0.07904942],
       [0.        , 0.23618328, 0.1432524 , 0.2865048 , 0.10539922,
        0.15809884],
       [0.        , 0.08688697, 0.05269961, 0.10539922, 0.03877421,
        0.05816131],
       [0.        , 0.13033046, 0.07904942, 0.15809884, 0.05816131,
        0.08724197]])

In [31]:
np.reshape(dRBF(np.array([[0.5], [[0]]]), X[:,11:14]),(-1,1), order='F') * np.reshape(dRBF(np.array([[0.5], [[0]]]), X[:,11:14]),(-1), order='F').T

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.1947002 , 0.11809164, 0.23618328, 0.08688697,
        0.13033046],
       [0.        , 0.11809164, 0.0716262 , 0.1432524 , 0.05269961,
        0.07904942],
       [0.        , 0.23618328, 0.1432524 , 0.2865048 , 0.10539922,
        0.15809884],
       [0.        , 0.08688697, 0.05269961, 0.10539922, 0.03877421,
        0.05816131],
       [0.        , 0.13033046, 0.07904942, 0.15809884, 0.05816131,
        0.08724197]])

In [ ]:




# # Testing expected derivative kernel vs derivative kernel in 1D
# xstar = np.arange(-5,5,0.5)

# plots_by_var = []
# for v in [0.01,1.0,2.0,3.0]:
#     plots_by_var.append(
#         plt_type.Scatter(x=np.squeeze(xstar), 
#                       y=np.squeeze(RBF_edK(np.array([[0]]),np.array([[v]]), xstar)), 
#                       mode='markers')
#     )
    

# plots_by_var.append(
#     plt_type.Scatter(x=np.squeeze(xstar), 
#                   y=np.squeeze(dRBF(np.atleast_2d(xstar) , np.array([[0]])).T), 
#                   mode='markers')
#     )
    
# plt(plots_by_var)